# PREDECIR CLIMA

In [24]:
import requests
from datetime import datetime, timedelta

API_KEY = "H8M4YDHTPDD73C6JESETFVZFY"
ubicacion = "Gustavo A. Madero, Mexico City, MX"

# Años que quieres descargar
años = [2024, 2025]

for año in años:
    for mes in range(9, 13):
        fecha_inicio = f"{año}-{mes:02d}-01"
        # calcular último día del mes
        if mes == 12:
            fecha_fin = f"{año}-12-31"
        else:
            fecha_fin = (datetime(año, mes + 1, 1) - timedelta(days=1)).strftime("%Y-%m-%d")
        
        print(f"📅 Descargando {año}-{mes:02d} ({fecha_inicio} → {fecha_fin})...")

        url = (
            f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
            f"{ubicacion}/{fecha_inicio}/{fecha_fin}"
            f"?unitGroup=metric&include=days&key={API_KEY}&contentType=csv"
        )

        respuesta = requests.get(url)

        if respuesta.status_code == 200:
            nombre_archivo = f"clima_cdmx_{año}_{mes:02d}.csv"
            with open(nombre_archivo, "wb") as f:
                f.write(respuesta.content)
            print(f"✅ Guardado {nombre_archivo}")
        else:
            print(f"❌ Error {respuesta.status_code} en {año}-{mes:02d}")


📅 Descargando 2024-09 (2024-09-01 → 2024-09-30)...
❌ Error 429 en 2024-09
📅 Descargando 2024-10 (2024-10-01 → 2024-10-31)...
❌ Error 429 en 2024-10
📅 Descargando 2024-11 (2024-11-01 → 2024-11-30)...
❌ Error 429 en 2024-11
📅 Descargando 2024-12 (2024-12-01 → 2024-12-31)...
❌ Error 429 en 2024-12
📅 Descargando 2025-09 (2025-09-01 → 2025-09-30)...
❌ Error 429 en 2025-09
📅 Descargando 2025-10 (2025-10-01 → 2025-10-31)...
❌ Error 429 en 2025-10
📅 Descargando 2025-11 (2025-11-01 → 2025-11-30)...
❌ Error 429 en 2025-11
📅 Descargando 2025-12 (2025-12-01 → 2025-12-31)...
❌ Error 429 en 2025-12


In [ ]:
import pandas as pd
from glob import glob

# --- CARGAR Y UNIR TODOS LOS AÑOS ---
archivos = glob("clima_cdmx_*.csv")  # busca todos los CSV que empiecen así
df_list = []

for archivo in archivos:
    data = pd.read_csv(archivo)
    df_list.append(data)

# Unir todos los dataframes
df = pd.concat(df_list, ignore_index=True)
print("✅ Datos combinados:", df.shape)

NameError: name 'lat' is not defined

In [6]:
# -------------------------------------------------------
# 🌤️ PREDICCIÓN DE TEMPERATURA EN GUSTAVO A. MADERO (CDMX)
# Basado en datos históricos (Visual Crossing)
# Entrenamiento: 2022–2024
# Predicción: día siguiente al último dato disponible
# -------------------------------------------------------

import pandas as pd
import numpy as np
from glob import glob
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# --- 1️⃣ CARGAR TODOS LOS ARCHIVOS CSV ---
archivos = sorted(glob("clima_cdmx_*.csv"))
dfs = []

for archivo in archivos:
    print(f"📂 Cargando {archivo} ...")
    df = pd.read_csv(archivo)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
print("✅ Datos combinados:", df.shape)

# --- 2️⃣ LIMPIEZA Y FORMATEO ---
df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
df = df.dropna(subset=["tempmax", "tempmin", "temp", "humidity", "windspeed"])

# --- 3️⃣ VARIABLES TEMPORALES ---
df["year"] = df["datetime"].dt.year
df["month"] = df["datetime"].dt.month
df["day"] = df["datetime"].dt.day
df["dayofyear"] = df["datetime"].dt.dayofyear

# --- 4️⃣ VARIABLES DE DÍA ANTERIOR ---
df["temp_avg_prev"] = df["temp"].shift(1)
df["tempmax_prev"] = df["tempmax"].shift(1)
df["tempmin_prev"] = df["tempmin"].shift(1)
df["humidity_prev"] = df["humidity"].shift(1)
df["windspeed_prev"] = df["windspeed"].shift(1)

# Eliminar filas con valores faltantes tras shift
df = df.dropna()

# --- 5️⃣ VARIABLES DE CICLO ANUAL (senos y cosenos) ---
df["sin_day"] = np.sin(2 * np.pi * df["dayofyear"] / 365)
df["cos_day"] = np.cos(2 * np.pi * df["dayofyear"] / 365)

# --- 6️⃣ DIVISIÓN ENTRE ENTRENAMIENTO Y PRUEBA ---
df_train = df[df["year"] < 2025]
df_test = df[df["year"] == 2025]

features = [
    "month", "day", "dayofyear",
    "tempmin", "humidity", "windspeed",
    "temp_avg_prev", "tempmax_prev", "tempmin_prev",
    "humidity_prev", "windspeed_prev",
    "sin_day", "cos_day"
]

X_train = df_train[features]
y_train = df_train["temp"]

X_test = df_test[features]
y_test = df_test["temp"]

# --- 7️⃣ ENTRENAR MODELO ---
print("\n🤖 Entrenando modelo RandomForest...")
modelo = RandomForestRegressor(n_estimators=400, random_state=42)
modelo.fit(X_train, y_train)

# --- 8️⃣ EVALUAR MODELO ---
y_pred = modelo.predict(X_test)
error = mean_absolute_error(y_test, y_pred)
print(f"📊 Error medio absoluto en 2025: {error:.2f} °C")

# --- 9️⃣ PREDICCIÓN PARA MAÑANA ---
ultima_fila = df.iloc[-1]
mañana = ultima_fila["datetime"] + timedelta(days=1)

entrada = pd.DataFrame([{
    "month": mañana.month,
    "day": mañana.day,
    "dayofyear": mañana.timetuple().tm_yday,
    "tempmin": ultima_fila["tempmin"],
    "humidity": ultima_fila["humidity"],
    "windspeed": ultima_fila["windspeed"],
    "temp_avg_prev": ultima_fila["temp"],
    "tempmax_prev": ultima_fila["tempmax"],
    "tempmin_prev": ultima_fila["tempmin"],
    "humidity_prev": ultima_fila["humidity"],
    "windspeed_prev": ultima_fila["windspeed"],
    "sin_day": np.sin(2 * np.pi * mañana.timetuple().tm_yday / 365),
    "cos_day": np.cos(2 * np.pi * mañana.timetuple().tm_yday / 365)
}])

prediccion = modelo.predict(entrada)[0]

print(f"\n🌡️ Temperatura estimada para {mañana.strftime('%Y-%m-%d')} en Gustavo A. Madero: {prediccion:.2f} °C")


📂 Cargando clima_cdmx_2022.csv ...
📂 Cargando clima_cdmx_2023.csv ...
📂 Cargando clima_cdmx_2024.csv ...
📂 Cargando clima_cdmx_2025.csv ...
✅ Datos combinados: (1461, 33)

🤖 Entrenando modelo RandomForest...
📊 Error medio absoluto en 2025: 0.67 °C

🌡️ Temperatura estimada para 2025-10-19 en Gustavo A. Madero: 19.32 °C


In [3]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# --- CARGAR DATOS ---
df = pd.read_csv("clima_cdmx_2024.csv")

# Asegurar que la fecha sea tipo datetime
df["datetime"] = pd.to_datetime(df["datetime"])

# --- LIMPIEZA ---
# Elimina filas con valores nulos
df = df.dropna(subset=["tempmax", "tempmin", "humidity", "windspeed"])

# --- CREAR VARIABLES TEMPORALES ---
df["year"] = df["datetime"].dt.year
df["month"] = df["datetime"].dt.month
df["day"] = df["datetime"].dt.day
df["dayofyear"] = df["datetime"].dt.dayofyear

# --- VARIABLES DE ENTRADA (X) Y OBJETIVO (y) ---
features = ["month", "day", "dayofyear", "tempmin", "humidity", "windspeed"]
X = df[features]
y = df["temp"]  # o “temp” si tienes la temperatura promedio del día

# --- DIVIDIR EN ENTRENAMIENTO Y PRUEBA ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- ENTRENAR MODELO ---
modelo = RandomForestRegressor(n_estimators=200, random_state=42)
modelo.fit(X_train, y_train)

# --- EVALUAR ---
y_pred = modelo.predict(X_test)
print("Error medio absoluto:", mean_absolute_error(y_test, y_pred))

# --- PREDICCIÓN PARA MAÑANA ---
mañana = datetime.now() + timedelta(days=1)
entrada = pd.DataFrame([{
    "month": mañana.month,
    "day": mañana.day,
    "dayofyear": mañana.timetuple().tm_yday,
    "tempmin": df["tempmin"].iloc[-1],  # usa últimos datos conocidos como base
    "humidity": df["humidity"].iloc[-1],
    "windspeed": df["windspeed"].iloc[-1]
}])

prediccion = modelo.predict(entrada)[0]
print(f"🌡️ Temperatura estimada para mañana ({mañana.strftime('%Y-%m-%d')}) a las 12:00 PM: {prediccion:.2f} °C")


Error medio absoluto: 0.7052905405405389
🌡️ Temperatura estimada para mañana (2025-10-27) a las 12:00 PM: 14.86 °C
